In [1]:
from edge_probing_utils import (
    JiantDatasetSingleSpan,
    JiantDatasetTwoSpan
    )
import edge_probing as ep
import torch
import torch.nn as nn
import torch.utils.data as data
from transformers import AutoModel, AutoTokenizer


**Setup:**

In [2]:
tasks = [
    #"ner", 
    #"semeval",
    "coref",
    #"sup-squad",
    #"ques",
    #"sup-babi",
    #"sup-hotpot",
    ]

task_type = {
    "ner": "single_span", 
    "semeval": "single_span",
    "coref": "two_span",
    "sup-squad": "two_span",
    "ques": "single_span",
    "sup-babi": "two_span",
    "sup-hotpot": "two_span",
    }

models = [
    "bert-base-uncased", 
    #"csarron/bert-base-uncased-squad-v1",
    ]

task_label_to_id = {
    "coref": {"0": 0, "1": 1},
    "sup-squad": {"0": 0, "1": 1},
    "sup-babi": {"0": 0, "1": 1},
    }

model_to_linestyle = {
    "bert-base-uncased": ":g", 
    "csarron/bert-base-uncased-squad-v1": "-y",
}

# Pick a dataset size: small, medium, big
size = "small"

In [3]:
import os
import json
import matplotlib.pyplot as plt

loss_function = nn.BCELoss()
batch_size = 32
num_layers = range(1,13,2)
num_workers = 0

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

# Disable warnings.
os.environ["TOKENIZERS_PARALLELISM"] = "false"

os.makedirs(f"../results/", exist_ok=True)
if os.path.isfile(f'../results/{size}.json'):
  with open(f'../results/{size}.json', 'r') as f:
    results = json.load(f)
else:
  results = {}

for model in models:
    tokenizer = AutoTokenizer.from_pretrained(model)
    for task in tasks:
        task_results = results.setdefault(task, {})
        label_to_id = task_label_to_id[task]
        train_data = ep.tokenize_jiant_dataset(
            tokenizer,
            *(ep.read_jiant_dataset(f"../data/{task}/{size}/train.jsonl")),
            label_to_id,
            max_seq_length=384
            )
        val_data = ep.tokenize_jiant_dataset(
            tokenizer,
            *(ep.read_jiant_dataset(f"../data/{task}/{size}/val.jsonl")),
            label_to_id,
            max_seq_length=384
            )
        if task_type[task] == "single_span":
            train_data = JiantDatasetSingleSpan(train_data)
            val_data = JiantDatasetSingleSpan(val_data)
        elif task_type[task] == "two_span":
            train_data = JiantDatasetTwoSpan(train_data)
            val_data = JiantDatasetTwoSpan(val_data)
        train_loader = data.DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=num_workers)
        val_loader = data.DataLoader(val_data, batch_size=batch_size, pin_memory=True, num_workers=num_workers)
        results[task][model] = ep.probing(
            train_loader,
            val_loader,
            val_loader,
            model,
            num_layers,
            loss_function,
            label_to_id,
            task_type[task],
            dev=device,
            epochs=1
            )

with open(f'../results/{size}.json', 'w') as f:
    json.dump(results, f)

the model
 12%|█▎        | 1/8 [00:00<00:01,  5.23it/s]Loss: 0.6864467561244965
Epoch 13
 12%|█▎        | 1/8 [00:00<00:01,  5.25it/s]Loss: 0.6863003671169281
Epoch 14
 12%|█▎        | 1/8 [00:00<00:01,  5.25it/s]Loss: 0.6846809387207031
Epoch 15
 12%|█▎        | 1/8 [00:00<00:01,  5.24it/s]Loss: 0.6845886707305908
Epoch 16
 12%|█▎        | 1/8 [00:00<00:01,  5.28it/s]Loss: 0.684779167175293
No improvement for 5 epochs, halving the learning rate to 2.5e-05
Epoch 17
 12%|█▎        | 1/8 [00:00<00:01,  5.25it/s]Loss: 0.6844232082366943
Epoch 18
 12%|█▎        | 1/8 [00:00<00:01,  5.23it/s]Loss: 0.6837808787822723
Epoch 19
 12%|█▎        | 1/8 [00:00<00:01,  5.25it/s]Loss: 0.6824261546134949
Epoch 20
 12%|█▎        | 1/8 [00:00<00:01,  5.26it/s]Loss: 0.6819027364253998
Epoch 21
 12%|█▎        | 1/8 [00:00<00:01,  5.26it/s]Loss: 0.6814058423042297
Epoch 22
 12%|█▎        | 1/8 [00:00<00:01,  5.27it/s]Loss: 0.6810083091259003
Epoch 23
 12%|█▎        | 1/8 [00:00<00:01,  5.24it/s]Loss: 0.680

KeyboardInterrupt: 

In [6]:
with open(f'../results/{size}.json', 'r') as f:
    results = json.load(f)
for task, task_results in results.items():
    for model, _ in task_results.items():
        ep.plot_task(results, task, model, model_to_linestyle[model], num_layers)
    plt.show()

KeyError: 1